In [13]:
import numpy as np # linear algebra
import pandas as pd
import xml.etree.ElementTree as ET
from pose_estimation.keypoints import Keypoints
from mediapipe.tasks.python.components.containers.landmark import Landmark
import mediapipe as mp

In [14]:
# Read XML File
xml_file = 'dataset/annotations.xml'
tree = ET.parse(xml_file)
root = tree.getroot()

dataset = {}

# Keypoint mapping from dataset numbers in annotations.xml to body part names used in Keypoints
datasetKeypoints = {
            5 : "left_shoulder",
            2 : "right_shoulder",
            6 : "left_elbow",
            3 : "right_elbow",
            7 : "left_wrist",
            4 : "right_wrist",
            11 : "left_hip",
            8 : "right_hip",
            12 : "left_knee",
            9 : "right_knee",
            13 : "left_ankle",
            10 : "right_ankle"
        }

images = root.findall("image")
for image in images:
    points = image.findall("points")
    keypoints = Keypoints(0, 0, 0, 0, 0 ,0 ,0 ,0 ,0 ,0 ,0 ,0, 0)
    for point in points:
        point_id = int(point.attrib["label"])
        x, y = point.attrib["points"].split(",")
        if point_id in datasetKeypoints:
            setattr(keypoints, datasetKeypoints[point_id], Landmark(x=float(x), y=float(y)))
    dataset.update({image.attrib["name"]: keypoints})

'''
Created a dictionary to store keypoints.
Format: {image_name: Keypoints}
Keypoints is our own Keypoints object.
'''

'\nCreated a dictionary to store keypoints.\nFormat: {image_name: Keypoints}\nKeypoints is our own Keypoints object.\n'

In [15]:
'''
Setup Media Pipe
'''
import mediapipe as mp
from pose_estimation.mediapipe import MediaPipe
from mediapipe.tasks.python import vision

media_pipe = MediaPipe()
media_pipe.initialize(mode=vision.RunningMode.IMAGE, parameter_path="/workspaces/Pose_Tracking/pose_landmarker_full.task")

In [32]:
def testImage(image_filename, image_keypoints):
    image = mp.Image.create_from_file("dataset/" + image_filename)
    estimatedKeypoints = media_pipe.process_image(image)
    correctKeypoints = image_keypoints
    estimatedKeypointsArray = estimatedKeypoints.to_numpy_positions()
    correctKeypointsArray = correctKeypoints.to_numpy_positions()
    for i in range(0, len(estimatedKeypointsArray)):
        print("Difference: ", compareKeypoints(estimatedKeypointsArray[i], correctKeypointsArray[i]))

def compareKeypoints(NormalizedLandmark1, NormalizedLandmark2):
    # TODO Implement this function
    return 0